**Import libraries**

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, when, col, to_date, round, count, avg
from datetime import datetime, timedelta

**Logging**

In [0]:
spark = SparkSession.builder.appName("Neo Analysis").getOrCreate()

# Logging 
def log_event(level, message):
    log_df = spark.createDataFrame([(datetime.now(), level, message)], ["timestamp", "level", "message"])
    log_df.write.format("delta").mode("append").saveAsTable("neo_logs")


**Gold layer function: build, add required objects/aggregation for Analytics)**

In [0]:
def neo_aggr():
    silver_df = spark.table("neo_approaches")

    gold_df = silver_df
    gold_df_v1 = silver_df.groupBy("hazard_level").agg(
        round(avg("miss_distance_km"), 2).alias("average_miss_distance"),
        count("*").alias("total_approaches"),
        avg("size_avg").alias("average_size")       
        )
    
     
    return gold_df, gold_df_v1

**Write Gold**

In [0]:
def write_gold(df, table_name):
    try:
        if df.isEmpty():
            log_event("INFO", f"No data to write to gold table: {table_name}")
            return
        df.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(table_name)
        log_event("INFO", f"Data written to gold table: {table_name}")
    except Exception as e:
        log_event("ERROR", f"Error writing data to gold table: {table_name}. Error: {e}")
        raise e

**Orchestrate the pipeline**

In [0]:
def run_neo_aggr():
    neo_gold, neo_gold_v1 = neo_aggr()
    write_gold(neo_gold, "neo_analysis")
    write_gold(neo_gold_v1, "neo_analysis_v1")

**Run the pipeline**

In [0]:
run_neo_aggr()